In [1]:
import os
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sympy import *

#### Part 1: Dataset Selection and Preprocessing (15 points)

Get the number of rows and columns from the first image in the directory

In [2]:
image = Image.open('yalefaces/subject01.centerlight')
nrows, ncolumns = image.height, image.width

Define a function to get load and preprocess the data

In [3]:
def process_images(image_directory):
    image_files = []
    for i, file in enumerate(os.listdir(image_directory)):
        image = Image.open(os.path.join(image_directory, file))
        image = image.resize((50, 50)).convert('L')
        image_files.append(np.asarray(image.getdata()))
    return image_files

#### Part 2: Eigenface Generation (20 points)

Load the images from the 'yalefaces' directory and flatten them into a dataset

In [4]:
X = np.asarray(process_images('yalefaces'))

Confirm the shape of the flattened dataset. Since the dataset contains 165 images and we have resized the images into 300 by 400 size, we should expect the shape to by 165 by 64 * 64

In [5]:
X.shape

(165, 2500)

Calculate the average face image

In [6]:
# Get the length of the images array
length_dataset = X.shape[0]

mean_face = np.sum(X, axis=0, dtype='float64') / length_dataset

Find the mean matrix using the numpy tile function. np.tile constructs an array by repeating a given dataset by the number of times specified

In [7]:
mean_matrix = np.tile(mean_face, (length_dataset, 1)) 

Mean-center the image data. Mean-centering calculates the difference between the mean value and the actual value, ensuring that we are only looking at the variance in the dataset

In [8]:
X_centered = X - mean_matrix

Caclulate the co-variance in the data. Convariance is a measure of the relationship between two variables.

In this case, we calculate the co-variance by finding the dot product of the image data and a transposed version of the image data

In [9]:
covariance_matrix = (X_centered.dot(X_centered.T)) / length_dataset

Find the eigenvalues and eigenvectors using the covariance matrix. Because they are square and symmetrical, covariance matrixes are diagonalizable, which means an eigendecomposition can be performed on the matrix.

In [10]:
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

In [11]:
eigenvalues

array([ 3.33326006e+06+0.00000000e+00j,  1.52599003e+06+0.00000000e+00j,
        9.56452718e+05+0.00000000e+00j,  8.07260102e+05+0.00000000e+00j,
        5.06382591e+05+0.00000000e+00j,  4.41204630e+05+0.00000000e+00j,
        2.90799269e+05+0.00000000e+00j,  2.39541432e+05+0.00000000e+00j,
        2.34898615e+05+0.00000000e+00j,  1.94322500e+05+0.00000000e+00j,
        1.37847598e+05+0.00000000e+00j,  1.16257401e+05+0.00000000e+00j,
        9.36427084e+04+0.00000000e+00j,  9.01826374e+04+0.00000000e+00j,
        8.77712605e+04+0.00000000e+00j,  8.04618289e+04+0.00000000e+00j,
        7.38324071e+04+0.00000000e+00j,  6.41477082e+04+0.00000000e+00j,
        6.22117346e+04+0.00000000e+00j,  5.97069618e+04+0.00000000e+00j,
        5.08778993e+04+0.00000000e+00j,  4.72798036e+04+0.00000000e+00j,
        4.26912174e+04+0.00000000e+00j,  4.03352813e+04+0.00000000e+00j,
        3.77239473e+04+0.00000000e+00j,  3.63773901e+04+0.00000000e+00j,
        3.16509969e+04+0.00000000e+00j,  2.91714546

Find the order of the eignevalues

In [12]:
order = eigenvalues.argsort()[::-1]

Sort the eigenvalues by their order

In [13]:
eigenvalues = eigenvalues[order]

Get the principal components

In [14]:
eigenvectors_C = eigenvectors[:, order]

Project all images to the eigen space

In [15]:
eigenvectors_C = X_centered.T @ eigenvectors 

Normalize all the eigenfaces

In [16]:
eigenfaces = preprocessing.normalize(eigenvectors_C.T)

ValueError: Complex data not supported
[[-6.58237087e+01+0.j -5.58298828e+01+0.j -5.43156059e+01+0.j ...
   3.51621356e+02+0.j  3.24839476e+02+0.j  3.04224719e+02+0.j]
 [ 1.14763727e+02+0.j  1.02401145e+02+0.j  9.83731149e+01+0.j ...
   3.60919810e+02+0.j  3.42864063e+02+0.j  3.36812258e+02+0.j]
 [-6.32229248e+01+0.j -6.36312392e+01+0.j -5.90238792e+01+0.j ...
  -5.18946391e+01+0.j -2.47224170e+01+0.j -1.38836523e+01+0.j]
 ...
 [ 4.32334374e-14+0.j  4.15876313e-14+0.j  3.77867484e-14+0.j ...
  -3.64170012e-14+0.j -9.33756207e-15+0.j  2.25933227e-14+0.j]
 [ 6.23550619e-14+0.j  9.04577065e-14+0.j  6.02001937e-14+0.j ...
  -2.13135699e-13+0.j -2.41041841e-13+0.j -2.38877967e-13+0.j]
 [ 6.21910876e-14+0.j  8.55395776e-14+0.j  6.36752607e-14+0.j ...
  -1.53167792e-13+0.j -1.84660219e-13+0.j -1.73713386e-13+0.j]]


Plot the first eigenface

In [ ]:
plt.imshow(np.resize(eigenfaces[0], (nrows,ncolumns)), cmap='gray')
plt.title('First Eigen Face')
plt.show()